In [15]:
import folium
from folium.plugins import HeatMap
import pandas as pd
import geopandas as gpd

# Load your dataset
mexico_dataset = pd.read_csv('../../data/mexico_combined_data.csv')

# Remove null values
mexico_dataset = mexico_dataset[mexico_dataset["latitude"].notna()]
mexico_dataset = mexico_dataset[mexico_dataset["longitude"].notna()]

# Fill missing values with mean
mexico_dataset["orgc"] = mexico_dataset["orgc"].fillna(mexico_dataset["orgc"].mean())

# Normalize the orgc values to emphasize differences
min_val = mexico_dataset["orgc"].min()
max_val = mexico_dataset["orgc"].max()
mexico_dataset["orgc_normalized"] = (mexico_dataset["orgc"] - min_val) / (max_val - min_val) * 2

# Load shapefile with polygon z geometry
shapefile = gpd.read_file("../../data/playa_viva/Playa_Viva_UoH data.shp")

# Calculate centroid of shapefile
centroid = shapefile.geometry.union_all().centroid

# Create a folium map centered on shapefile centroid
m = folium.Map(location=[centroid.y, centroid.x], zoom_start=12)

# Prepare heatmap data
heat_data = list(zip(mexico_dataset["latitude"], mexico_dataset["longitude"], mexico_dataset["orgc_normalized"]))
HeatMap(heat_data, min_opacity=0.2, radius=15).add_to(m)

# Add shapefile polygons to the map
for _, row in shapefile.iterrows():
    # Create popup HTML with all column information
    popup_html = "<table>"
    for col in row.index:
        if col != 'geometry':  # Skip geometry column
            popup_html += f"<tr><th>{col}</th><td>{row[col]}</td></tr>"
    popup_html += "</table>"
    
    folium.GeoJson(
        row["geometry"],
        name=row["name"] if "name" in row else "Polygon",
        style_function=lambda x: {
            'fillColor': '#ffaf00',
            'color': '#000000',
            'weight': 1,
            'fillOpacity': 0.5
        },
        popup=folium.Popup(popup_html, max_width=300)
    ).add_to(m)

# Display map
m